<a href="https://colab.research.google.com/github/vendo495/projects/blob/main/Machine_learning(book_rental_recommendations).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
books = pd.read_csv('/content/BX-Books.csv', sep=',', error_bad_lines=False, encoding="latin-1")
books = books[['isbn', 'book_title', 'book_author', 'year_of_publication', 'publisher']]
print(books.head())


<ipython-input-67-bcfbeaf060a4>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv('/content/BX-Books.csv', sep=',', error_bad_lines=False, encoding="latin-1")


        isbn                                         book_title  \
0  195153448                                Classical Mythology   
1    2005018                                       Clara Callan   
2   60973129                               Decision in Normandy   
3  374157065  Flu: The Story of the Great Influenza Pandemic...   
4  393045218                             The Mummies of Urumchi   

            book_author year_of_publication                   publisher  
0    Mark P. O. Morford                2002     Oxford University Press  
1  Richard Bruce Wright                2001       HarperFlamingo Canada  
2          Carlo D'Este                1991             HarperPerennial  
3      Gina Bari Kolata                1999        Farrar Straus Giroux  
4       E. J. W. Barber                1999  W. W. Norton &amp; Company  


<ipython-input-67-bcfbeaf060a4>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('/content/BX-Books.csv', sep=',', error_bad_lines=False, encoding="latin-1")


In [ ]:
# 2. Clean up NaN values
books = books.dropna()

In [ ]:
# 3. Read the data where ratings are given by users
ratings = pd.read_csv('/content/BX-Book-Ratings.csv', sep=None, engine='python', nrows=10)
ratings


,user_id,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6
5,276733,2080674722,0
6,276736,3257224281,8
7,276737,600570967,6
8,276744,038550120X,7
9,276745,342310538,10


In [ ]:
# 4. Take a quick look at the number of unique users and books
unique_users = ratings['user_id'].nunique()
unique_books = ratings['isbn'].nunique()
print(f"Number of unique users: {unique_users}")
print(f"Number of unique books: {unique_books}")

Number of unique users: 9
Number of unique books: 10


In [ ]:
# 5. Convert ISBN variables to numeric numbers in the correct order
isbn_mapping = {isbn: i for i, isbn in enumerate(ratings['isbn'].unique())}
ratings['ISBN_Numeric'] = ratings['isbn'].map(isbn_mapping)


In [ ]:
# 6. Convert the user_id variable to numeric numbers in the correct order
user_id_mapping = {user_id: i for i, user_id in enumerate(ratings['user_id'].unique())}
ratings['User_ID_Numeric'] = ratings['user_id'].map(user_id_mapping)


In [ ]:
# 7. Convert both user_id and ISBN to the ordered list, i.e., from 0...n-1
# This step has been achieved in steps 5 and 6


In [ ]:
# 8. Re-index the columns to build a matrix
user_item_matrix = ratings.pivot(index='User_ID_Numeric', columns='ISBN_Numeric', values='rating')


In [ ]:
# 9. Split your data into two sets (training and testing)
def train_test_split(matrix, test_ratio=0.2):
    test_mask = np.random.rand(*matrix.shape) < test_ratio
    train_mask = ~test_mask
    train_matrix = matrix.copy()
    test_matrix = matrix.copy()
    train_matrix[test_mask] = np.nan
    test_matrix[train_mask] = np.nan
    return train_matrix, test_matrix
train_matrix, test_matrix = train_test_split(user_item_matrix)


In [ ]:
# 10. Make predictions based on user and item variables
def compute_similarity(matrix):
    matrix_filled = matrix.fillna(0)
    sim = matrix_filled.dot(matrix_filled.T)
    norms = np.array([np.sqrt(np.diagonal(sim))])
    similarity = sim / norms / norms.T
    return similarity

In [ ]:
def predict_ratings_simple(user_similarity, user_item_matrix):
    weighted_sum = np.dot(user_similarity, user_item_matrix.fillna(0))
    sum_of_similarities = np.abs(user_similarity).sum(axis=1)
    sum_of_similarities[sum_of_similarities == 0] = 1
    predicted_ratings = weighted_sum / sum_of_similarities[:, np.newaxis]
    return pd.DataFrame(predicted_ratings, index=user_item_matrix.index, columns=user_item_matrix.columns)

user_similarity = compute_similarity(train_matrix)
predicted_ratings = predict_ratings_simple(user_similarity, train_matrix)


<ipython-input-78-cb0b4c645107>:5: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predicted_ratings = weighted_sum / sum_of_similarities[:, np.newaxis]


In [ ]:
# 11. Use RMSE to evaluate the predictions
def compute_rmse(y_true, y_pred):
    actual_ratings = y_true[y_true.notna()]
    predicted_for_actual = y_pred[y_true.notna()]
    rmse = np.sqrt(((actual_ratings - predicted_for_actual) ** 2).mean())
    return rmse

rmse_value = compute_rmse(test_matrix, predicted_ratings)
print(f"RMSE: {rmse_value}")

RMSE: ISBN_Numeric
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
6   NaN
7   NaN
8   NaN
9   NaN
dtype: float64
